<a href="https://colab.research.google.com/github/gabibu/nlp/blob/master/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2

This assignment is about training and evaluating a POS tagger with some real data. The dataset is available through the Universal Dependencies (https://universaldependencies.org/) (UD) project. To get to know the project, please visit https://universaldependencies.org/introduction.html)

In [6]:
import numpy as np
import operator
import nltk

!pip install conllutils
!pip install conll_df
import conllutils

  ERROR: Failed building wheel for conll-df
  Running setup.py clean for conll-df
Failed to build conll-df
    Running setup.py install for conll-df ... done


**Part 1** (getting the data)

You can download the dataset files directly from the UD website, but it will let you only download all the languages in one compressed file. In this assignment you will be working with th GUM dataset, which you can download directly from:
https://github.com/UniversalDependencies/UD_English-GUM.
Please download it to your colab machine.



In [2]:
!git clone https://github.com/UniversalDependencies/UD_English-GUM

Cloning into 'UD_English-GUM'...
remote: Enumerating objects: 2417, done.
remote: Counting objects: 100% (1428/1428), done.
remote: Compressing objects: 100% (656/656), done.
remote: Total 2417 (delta 1245), reused 949 (delta 772), pack-reused 989
Receiving objects: 100% (2417/2417), 16.19 MiB | 9.46 MiB/s, done.
Resolving deltas: 100% (2135/2135), done.


We will use the (train/dev/test) files:

UD_English-GUM/en_gum-ud-train.conllu

UD_English-GUM/en_gum-ud-dev.conllu

UD_English-GUM/en_gum-ud-test.conllu

They are all formatted in the conllu format. You may read about it [here](https://universaldependencies.org/format.html). There is a utility library **conllutils**, which can help you read the data into the memory. It has already been installed and imported above.

You should write a code that reads the three datasets into memory. You may choose the data structure by yourself. As you can see, every word is represented by a line, with columns representing specific features. We are only interested in the first and fourth columns, corresponding to the word and its POS tag.

In [50]:
from conllutils import pipe

NUM_REGEX = r'[0-9]+|[0-9]+\.[0-9]+|[0-9]+[0-9,]+'
NUM_FORM = '__number__'

p = pipe()
p.only_words()
p.only_universal_deprel()
p.upos_feats()
p.lowercase('form')
p.replace('form', NUM_REGEX, NUM_FORM)

train_file = 'UD_English-GUM/en_gum-ud-train.conllu'
indexed_fields = {'form', 'upos_feats', 'deprel'}

index = pipe().read_conllu(train_file).pipe(p).create_index(fields=indexed_fields)
train_data = pipe().read_conllu(train_file).pipe(p).to_instance(index).collect()


train_data[-10:-1]

[{'deprel': array([ 8,  3,  9, 13, 12, 10,  2,  3,  7,  1]),
  'form': array([5824,    1, 4052,   32, 5411,  435,   20,    1, 2882,    3]),
  'head': array([0, 3, 1, 5, 3, 1, 9, 9, 1, 1]),
  'upos_feats': array([29,  6,  7,  8,  7,  9,  3,  6,  7,  1])},
 {'deprel': array([ 8,  3,  9, 13, 12,  2,  3,  7,  1, 17, 14, 10, 17,  1]),
  'form': array([ 652,    1,  233,    5, 1804,    6,    1, 1490,    2, 5948,  201,
         1663, 4358,    3]),
  'head': array([ 0,  3,  1,  5,  3,  8,  8,  1, 10,  1, 13, 13, 10,  1]),
  'upos_feats': array([29,  6,  2,  8,  2,  3,  6,  2,  1, 17, 16,  9, 14,  1])},
 {'deprel': array([ 4, 15,  8, 10, 10,  2, 23,  5,  1]),
  'form': array([  26,  110,  134,   99,  531,  104,    9, 1294,    3]),
  'head': array([3, 3, 0, 5, 3, 8, 8, 5, 3]),
  'upos_feats': array([42, 20, 10, 70, 64,  3, 12,  7,  1])},
 {'deprel': array([ 8, 23,  9,  2,  5, 13, 23, 12,  2,  5,  2,  3,  7, 13, 12,  2,  3,
          7,  2,  6,  6,  7,  1]),
  'form': array([ 652,   52,  761,    4

In [63]:
# Your code goes here
from conllutils import pipe
train_file = "UD_English-GUM/en_gum-ud-train.conllu"
# p = pipe()

p = pipe().read_conllu(train_file)
[[(t.form, t.upos) for t in s] for s in p.collect()[0:3]]

[[('Aesthetic', 'ADJ'),
  ('Appreciation', 'NOUN'),
  ('and', 'CCONJ'),
  ('Spanish', 'ADJ'),
  ('Art', 'NOUN'),
  (':', 'PUNCT')],
 [('Insights', 'NOUN'), ('from', 'ADP'), ('Eye-Tracking', 'NOUN')],
 [('Claire', 'PROPN'),
  ('Bailey-Ross', 'PROPN'),
  ('claire.bailey-ross@port.ac.uk', 'PROPN'),
  ('University', 'PROPN'),
  ('of', 'ADP'),
  ('Portsmouth', 'PROPN'),
  (',', 'PUNCT'),
  ('United', 'PROPN'),
  ('Kingdom', 'PROPN')]]

{'upos_feats': array([ 4, 25, 14,  3,  4,  1,  4,  1]), 'deprel': array([ 4, 15,  8,  2,  7,  1, 22,  1]), 'feats': array([ 1, 16, 10, -1,  1, -1,  1, -1]), 'lemma': array([ 751,    4,  265,    9, 2365,    2, 4402,    3]), 'misc': array([13859,    -1,    -1,    -1,  3783,    -1,  8601,    -1]), 'xpos': array([ 4, 13, 16,  2,  4,  7,  4,  8]), 'form': array([ 721,   15,  281,    7, 2439,    2, 4800,    3]), 'head': array([3, 3, 0, 5, 3, 7, 5, 3]), 'upos': array([6, 9, 3, 4, 6, 2, 6, 2])}


In [57]:
x[x['w'] == 'prevalence']

,s,i,w,l,x,p,g,f,s_type,speaker,Tense,Gender,Mood,Definite,Poss,Entity,Degree,Polarity,Person,Number,Case,Discourse,Reflex,Type,type
41692,2054,11,prevalence,prevalence,NOUN,NN,9,conj,decl,_,_,_,_,_,_,_,_,_,_,Sing,_,_,_,_,_


In [64]:
x = df.reset_index()
w = []
cc= 0
for ii, row in x.iterrows():
  word = row['w']
  index = row['i']
  pos = row['x']
  if index == 1:
    print(w)
    cc += 1
    if cc > 3:
      break
    
    w = []
    w.append((word, index))
  else:
    w.append((word, index))
  

[]
[('Aesthetic', 1), ('Appreciation', 2), ('and', 3), ('Spanish', 4), ('Art', 5), (':', 6)]
[('Insights', 1), ('from', 2), ('Eye-Tracking', 3)]
[('Claire', 1), ('Bailey-Ross', 2), ('claire.bailey-ross@port.ac.uk', 3), ('University', 4), ('of', 5), ('Portsmouth', 6), (',', 7), ('United', 8), ('Kingdom', 9)]


We will use the (train/dev/test) files:

UD_English-GUM/en_gum-ud-train.conllu

UD_English-GUM/en_gum-ud-dev.conllu

UD_English-GUM/en_gum-ud-test.conllu

They are all formatted in the conllu format. You may read about it [here](https://universaldependencies.org/format.html). There is a utility library **conllutils**, which can help you read the data into the memory. It has already been installed and imported above.

You should write a code that reads the three datasets into memory. You may choose the data structure by yourself. As you can see, every word is represented by a line, with columns representing specific features. We are only interested in the first and fourth columns, corresponding to the word and its POS tag.

We will use the (train/dev/test) files:

UD_English-GUM/en_gum-ud-train.conllu

UD_English-GUM/en_gum-ud-dev.conllu

UD_English-GUM/en_gum-ud-test.conllu

They are all formatted in the conllu format. You may read about it [here](https://universaldependencies.org/format.html). There is a utility library **conllutils**, which can help you read the data into the memory. It has already been installed and imported above.

You should write a code that reads the three datasets into memory. You may choose the data structure by yourself. As you can see, every word is represented by a line, with columns representing specific features. We are only interested in the first and fourth columns, corresponding to the word and its POS tag.

**Part 2**

Write a class **simple_tagger**, with methods *train* and *evaluate*. The method *train* receives the data as a list of sentences, and use it for training the tagger. In this case, it should learn a simple dictionary that maps words to tags, defined as the most frequent tag for every word (in case there is more than one most frequent tag, you may select one of them randomly). The dictionary should be stored as a class member for evaluation.

The method *evaluate* receives the data as a list of sentences, and use it to evaluate the tagger performance. Specifically, you should calculate the word and sentence level accuracy.
The evaluation process is simply going word by word, querying the dictionary (created by the train method) for each word’s tag and compare it to the true tag of that word. The word-level accuracy is the number of successes divided by the number of words. For OOV (out of vocabulary, or unknown) words, the tagger should assign the most frequent tag in the entire training set (i.e., the mode). The function should return the two numbers: word level accuracy and sentence level accuracy.


In [ ]:
class simple_tagger:
  def train(self, data):
    # TODO
  
  def evaluate(self, data):
    # TODO

**Part 3**

Similar to part 2, write the class hmm_tagger, which implements HMM tagging. The method *train* should build the matrices A, B and Pi, from the data as discussed in class. The method *evaluate* should find the best tag sequence for every input sentence using he Viterbi decoding algorithm, and then calculate the word and sentence level accuracy using the gold-standard tags. You should implement the Viterbi algorithm in the next block and call it from your class.

Additional guidance:
1. The matrix B represents the emissions probabilities. Since B is a matrix, you should build a dictionary that maps every unique word in the corpus to a serial numeric id (starting with 0). This way columns in B represents word ids.
2. During the evaluation, you should first convert each word into it’s index and then create the observation array to be given to Viterbi, as a list of ids. OOV words should be assigned with a random tag. To make sure Viterbi works appropriately, you can simply break the sentence into multiple segments every time you see an OOV word, and decode every segment individually using Viterbi.


In [ ]:
class hmm_tagger:
  def train(self, data):
    # TODO

  def evaluate(self, data):
    # TODO

In [ ]:
# Viterbi
def viterbi (observations, A, B, Pi):
  #...

  return best_sequence

# A simple example to run the Viterbi algorithm:
#( Same as in presentation "NLP 3 - Tagging" on slide 35)

# A = np.array([[0.3, 0.7], [0.2, 0.8]])
# B = np.array([[0.1, 0.1, 0.3, 0.5], [0.3, 0.3, 0.2, 0.2]])
# Pi = np.array([0.4, 0.6])
# print(viterbi([0, 3, 1, 0], A, B, Pi))
# Expected output: 1, 1, 1, 1

**Part 4**

Compare the results obtained from both taggers and a MEMM tagger, implemented by NLTK (a known NLP library), over the test data. To train the NLTK MEMM tagger you should execute the following lines (it may take some time to train...):

In [ ]:
from nltk.tag import tnt 

tnt_pos_tagger = tnt.TnT()
tnt_pos_tagger.train(train_data)
print(tnt_pos_tagger.evaluate(test_data))

Print both, word level and sentence level accuracy for all the three taggers in a table.

In [ ]:
# Your code goes here